In [15]:
!git clone https://github.com/lhbeom/data.git

fatal: destination path 'data' already exists and is not an empty directory.


In [16]:
# 모듈 불러오기
import pandas as pd
import os
import time
import copy
import random
import pickle
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

# %% library
import gc
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from easydict import EasyDict as edict
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler

import os

import numpy as np
import pandas as pd
import torch
import torch.nn as nn




from easydict import EasyDict as edict
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from xgboost.sklearn import XGBRegressor

pd.set_option('display.max_columns', 100)

In [17]:
data_path="/content/data/sort_top100_공유.csv"


user_list = pd.read_csv(data_path)['User'].unique()
data=pd.read_csv(data_path)
data=data.loc[data.User.isin(user_list)]
data=data.sort_values(["User", "Date"], ascending=(True, True))
gc.collect()
data["Date"]=pd.to_datetime(data["Date"])
data['Date'] += pd.Timedelta(hours=9)
data['hour'] = np.digitize(data['Date'].dt.hour, [0, 5, 9, 17, 21, 24])  # 9시 추가 및 시간 --> 새벽,아침,오후,저녁으로 구분
seasons = np.digitize(data['Date'].dt.month, [0, 3, 6, 9, 12])  # 월별 --> 겨울, 봄, 여름, 가을 ,겨울로 구분 , change from 5 to 1
data['Season'] = np.where(seasons == 5, 1, seasons)
data.drop(['Date'], axis=1, inplace=True)
data.reset_index(drop=True,inplace=True)

user_with_data = data.copy()
X_col_continious_names = ["loadLevel", "DT_init", "Init_Display_time"]
X_col_ex_continious_names = ["Season", "hour","User"]
y_col_names = ["Running_time"]
X_continious, X_ex_continious, y = user_with_data[X_col_continious_names], user_with_data[X_col_ex_continious_names], \
                                   user_with_data[y_col_names]
X_ex_continious = pd.get_dummies(X_ex_continious, columns=X_col_ex_continious_names, drop_first=True)
# y=y[y.columns[0]]

x_col_time_names=[f"{type}_{time}" for type in ["DT","CT","Hum"] for time in list(range(5,55,5))]
X_time_continious =  user_with_data[x_col_time_names]
X = pd.concat([X_continious, X_time_continious, X_ex_continious], axis=1)


train_x, test_x, train_y, test_y= train_test_split(X,y, test_size=0.20, random_state=0)
train_x, valid_x, train_y, valid_y= train_test_split(train_x,train_y, test_size=0.25, random_state=0)

scaler = StandardScaler()
train_x.loc[:,X_col_continious_names] = scaler.fit_transform(train_x[X_col_continious_names])
test_x.loc[:,X_col_continious_names] = scaler.transform(test_x[X_col_continious_names])
valid_x.loc[:,X_col_continious_names] = scaler.transform(valid_x[X_col_continious_names])


train_y=train_y.values.tolist()
valid_y=valid_y.values.tolist()
test_y=test_y.values.tolist()

In [18]:
%%time
def Data_for_LSTM(X1, n_inner_split):
    X=X1
    x_col_time_DT_0=[f"{type}_{time}" for type in ["DT"] for time in list(range(5,55,5))]
    x_col_time_CT_0=[f"{type}_{time}" for type in ["CT"] for time in list(range(5,55,5))]
    x_col_time_Hum_0=[f"{type}_{time}" for type in ["Hum"] for time in list(range(5,55,5))]

    x_col_time_DT=[f"{type}_{time}" for type in ["DT"] for time in list(range(0,55,5))]
    x_col_time_CT=[f"{type}_{time}" for type in ["CT"] for time in list(range(0,55,5))]
    x_col_time_Hum=[f"{type}_{time}" for type in ["Hum"] for time in list(range(0,55,5))]



    cols_continuous=["DT","CT","Hum"]
    cols=list(X1.columns)
    cols_Needed = cols_continuous
    cols_Needed.append("DT_init")
    cols_Needed.extend(x_col_time_DT_0)
    cols_Needed.extend(x_col_time_CT_0)
    cols_Needed.extend(x_col_time_Hum_0)
    for i in (cols_Needed):
        if i in cols:
            cols.remove(i)
    nn=n_inner_split
    ###########################################################################3
    Jlist=np.linspace(0,(X.shape[0]//nn)*nn,X.shape[0]//nn+1)
    Jlist=list(Jlist.astype(int))
    Jlist.append(X.shape[0])
    print(Jlist)

    for ii in range(0,len(Jlist)-1):
        j=Jlist[ii]
        print(j)
        

        s1= X1.iloc[j][x_col_time_DT_0]
        s2= X1.iloc[j][x_col_time_CT_0]
        s3= X1.iloc[j][x_col_time_Hum_0]

        s1["DT_0"]=X1.iloc[j]["DT_init"]
        s2["CT_0"]=X1.iloc[j]["CT_5"]
        s3["Hum_0"]=X1.iloc[j]["Hum_5"]

        s1=s1.reindex(x_col_time_DT)
        s1.index=range(0,55,5)
        s2=s2.reindex(x_col_time_CT)
        s2.index=range(0,55,5)
        s3=s3.reindex(x_col_time_Hum)
        s3.index=range(0,55,5)
        a1=pd.concat([s1,s2,s3],axis=1)
        a1.columns=['DT',"CT","Hum"]

        b=pd.DataFrame(X1.iloc[j][cols].values)
        b=b.transpose()
        b.columns=cols
        a1=pd.concat([a1,b],axis=1)

        a1.fillna(method='ffill',inplace=True)


        globals()['DATA_{}'.format(ii)] = np.array(a1).reshape(1,a1.shape[0],a1.shape[1])


        for k in range(j+1,Jlist[ii+1]):
            if(k%1000==1):
                print(k)
            s1= X1.iloc[k][x_col_time_DT_0]
            s2= X1.iloc[k][x_col_time_CT_0]
            s3= X1.iloc[k][x_col_time_Hum_0]

            s1["DT_0"]=X1.iloc[k]["DT_init"]
            s2["CT_0"]=X1.iloc[k]["CT_5"]
            s3["Hum_0"]=X1.iloc[k]["Hum_5"]

            s1=s1.reindex(x_col_time_DT)
            s1.index=range(0,55,5)
            s2=s2.reindex(x_col_time_CT)
            s2.index=range(0,55,5)
            s3=s3.reindex(x_col_time_Hum)
            s3.index=range(0,55,5)
            a1=pd.concat([s1,s2,s3],axis=1)
            a1.columns=['DT',"CT","Hum"]

            b=pd.DataFrame(X1.iloc[k][cols].values)
            b=b.transpose()
            b.columns=cols
            a1=pd.concat([a1,b],axis=1)

            a1.fillna(method='ffill',inplace=True)

            globals()['DATA_{}'.format(ii)] = np.concatenate([globals()['DATA_{}'.format(ii)],np.array(a1).reshape(1,a1.shape[0],a1.shape[1])],axis=0)


    ######################################################################################################
    DATA=np.concatenate([globals()['DATA_{}'.format(q)] for q in range(0,ii+1)],axis=0)
    return DATA

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 10 µs


In [19]:
train_X_l=Data_for_LSTM(train_x,1000)
# train_X_l=Data_for_LSTM(X,1000)
test_X_l=Data_for_LSTM(test_x,1000)
valid_X_l=Data_for_LSTM(valid_x,1000)

import pickle

# 데이터 저장
with open('Data_lstm_train.pkl', 'wb') as f:
	pickle.dump(train_X_l, f, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('Data_lstm_test.pkl', 'wb') as f:
	pickle.dump(test_X_l, f, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('Data_lstm_val.pkl', 'wb') as f:
	pickle.dump(valid_X_l, f, protocol=pickle.HIGHEST_PROTOCOL)

[0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000, 21000, 22000, 22392]
0
1
1000
1001
2000
2001
3000
3001
4000
4001
5000
5001
6000
6001
7000
7001
8000
8001
9000
9001
10000
10001
11000
11001
12000
12001
13000
13001
14000
14001
15000
15001
16000
16001
17000
17001
18000
18001
19000
19001
20000
20001
21000
21001
22000
22001
[0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 7465]
0
1
1000
1001
2000
2001
3000
3001
4000
4001
5000
5001
6000
6001
7000
7001
[0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 7465]
0
1
1000
1001
2000
2001
3000
3001
4000
4001
5000
5001
6000
6001
7000
7001


In [20]:
import pickle

# 데이터 로드
with open('Data_lstm_train.pkl', 'rb') as f:
	train_X_l1=pickle.load(f)
    
with open('Data_lstm_test.pkl', 'rb') as f:
	test_X_l1=pickle.load(f)
    
with open('Data_lstm_val.pkl', 'rb') as f:
	valid_X_l1=pickle.load(f)

In [28]:
# Hyperparameter setting
data_dir = '/content/LG_time_series_day11/input/har-data'
batch_size = 32
num_classes = 6
num_epochs = 200
# window_size = 50 # 타임라인 크기
window_size = 3 # 타임라인 크기
input_size = 111
hidden_size = 400 # 히든레이어의 노드 개수 개념
num_layers = 2
bidirectional = True

random_seed = 42
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Detect if we have a GPU available

In [29]:
# seed 고정
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [30]:
def create_classification_dataset(Train, Valid, Test, time_range, batch_size):
    print(len(Train))
    train_x, valid_x, test_x = Train[0], Valid[0], Test[0]
    train_y, valid_y, test_y = Train[1], Valid[1], Test[1]

    # train data를 시간순으로 8:2의 비율로 train/validation set으로 분할
    train_x=train_x[:,:time_range,:]
    valid_x=valid_x[:,:time_range,:]
    test_x=test_x[:,:time_range,:]
   

    
    print(train_x.shape)
    # print(type(x_train))
    # # train/validation/test 데이터를 window_size 시점 길이로 분할
    # datasets = []
    # for set in [(x_train, y_train, n_train), (x_valid, y_valid, n_valid), (x_test, y_test, n_test,2)]:
    #     T = set[0].shape[-1]
    #     print(len(set))
    #     print(x_train.shape, x_valid.shape,x_test.shape)
    #     print(set[0].shape, set[1].shape,set[2])
    #     windows = np.split(set[0][:, :, :window_size * (T // window_size)], (T // window_size), -1)
    #     print("전",len(windows))
    #     windows = np.concatenate(windows, 0)
    #     print("후",len(windows))
    #     labels = np.split(set[1][:, :window_size * (T // window_size)], (T // window_size), -1)
    #     print("전1",len(labels[0][4]))
    #     labels = np.round(np.mean(np.concatenate(labels, 0), -1))
    #     print("후1",len(labels))
    #     datasets.append(torch.utils.data.TensorDataset(torch.Tensor(windows), torch.Tensor(labels)))

    # # train/validation/test DataLoader 구축
    # print("후1",len(datasets[0]),len(datasets[1]),len(datasets[2]))
    # 
    
    # train/validation/test 데이터를 window_size 시점 길이로 분할
    datasets = []
    for set in [(train_x, train_y), (valid_x, valid_y), (test_x, test_y)]:
        datasets.append(torch.utils.data.TensorDataset(torch.Tensor(set[0]), torch.Tensor(set[1])))

    trainset, validset, testset = datasets[0], datasets[1], datasets[2]
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(validset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)


    return train_loader, valid_loader, test_loader
  

In [31]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers,  bidirectional, rnn_type='rnn'):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn_type = rnn_type
        self.num_directions = 2 if bidirectional == True else 1
        
        # rnn_type에 따른 recurrent layer 설정
        if self.rnn_type == 'rnn':
            self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional)
        elif self.rnn_type == 'lstm':
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional)
        elif self.rnn_type == 'gru':
            self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional)
        
        # bidirectional에 따른 fc layer 구축
        # bidirectional 여부에 따라 hidden state의 shape가 달라짐 (True: 2 * hidden_size, False: hidden_size)
        self.fc = nn.Linear(self.num_directions * hidden_size, 1)

    def forward(self, x):
        # data dimension: (batch_size x input_size x seq_len) -> (batch_size x seq_len x input_size)로 변환
        # x = torch.transpose(x, 1, 2)
        
        # initial hidden states 설정
        h0 = torch.zeros(self.num_directions * self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # 선택한 rnn_type의 RNN으로부터 output 도출
        if self.rnn_type in ['rnn', 'gru']:
            out, h_n = self.rnn(x, h0)  # out: tensor of shape (batch_size, seq_length, hidden_size)
        else:
            # initial cell states 설정
            c0 = torch.zeros(self.num_directions * self.num_layers, x.size(0), self.hidden_size).to(device)
            out, _ = self.rnn(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        out = self.fc(out[:, -1, :]) #output 중 seq의 마지막 시점
        # out = self.fc(torch.transepose(h_n[-1, :, :],0,1)) #h_n에서도 가져 올 수 있으나 더 복잡하다
        return out

In [32]:
# LSTM 모델 구축

lstm = RNN(input_size, hidden_size, num_layers, bidirectional, rnn_type='lstm')
lstm = lstm.to(device)
print(lstm)

RNN(
  (rnn): LSTM(111, 400, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=800, out_features=1, bias=True)
)


In [33]:
# GRU 모델 구축
gru = RNN(input_size, hidden_size, num_layers, bidirectional, rnn_type='gru')
gru = gru.to(device)
print(gru)

RNN(
  (rnn): GRU(111, 400, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=800, out_features=1, bias=True)
)


## 3.Training

In [34]:
def train_model(model, dataloaders, criterion, num_epochs, modelType, optimizer):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss= np.inf

    for epoch in range(num_epochs):
        if(epoch%50 == 0):
            print('Epoch {}/{}'.format(epoch + 1, num_epochs))
            print('-' * 10)

        # 각 epoch마다 순서대로 training과 validation을 진행
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 모델을 training mode로 설정
            else:
                model.eval()   # 모델을 validation mode로 설정

            running_loss = 0.0
            running_loss2= 0.0
            running_total = 0

            # training과 validation 단계에 맞는 dataloader에 대하여 학습/검증 진행
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device, dtype=torch.float32)

                # parameter gradients를 0으로 설정
                optimizer.zero_grad()

                # forward
                # training 단계에서만 gradient 업데이트 수행
                with torch.set_grad_enabled(phase == 'train'):
                    # input을 model에 넣어 output을 도출한 후, loss를 계산함
                    outputs = model(inputs)
                    loss = criterion(outputs.to(torch.float32), labels.to(torch.float32))

                    # output 중 최댓값의 위치에 해당하는 class로 예측을 수행
                    _, preds = torch.max(outputs, 1)

                    # backward (optimize): training 단계에서만 수행
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # batch별 loss를 축적함
                running_loss += loss.item() * inputs.size(0)
                running_loss2 += mean_absolute_percentage_error(labels, outputs) * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                running_total += labels.size(0)

            # epoch의 loss 및 accuracy 도출
            epoch_loss = running_loss / running_total
            epoch_loss2 = running_loss2 / running_total

            if(epoch%50 == 0):
                print('{} Loss: {:.4f} MAPE: {:.4f}'.format(phase, epoch_loss, epoch_loss2))

            # validation 단계에서 validation loss가 감소할 때마다 best model 가중치를 업데이트함
            if phase == 'val' and epoch_loss < best_loss :
                best_loss  = epoch_loss 
                best_loss2 =  epoch_loss2
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(best_model_wts, './best_model_{}.pt'.format(modelType))
            if phase == 'val':
                val_acc_history.append(epoch_loss)

        print()

    # 전체 학습 시간 계산
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val MAE / MAPE: {:4f} / {:4f}'.format(best_loss, best_loss2))

    # validation loss가 가장 낮았을 때의 best model 가중치를 불러와 best model을 구축함
    model.load_state_dict(best_model_wts)
#     torch.save(model, './best_model_{}.pt'.format(modelType))
    
    # best model 가중치 저장
    # torch.save(best_model_wts, '../output/best_model.pt')
    return model, val_acc_history

In [14]:
# trining 단계에서 사용할 Dataloader dictionary 생성
dataloaders_dict = {
    'train': train_loader,
    'val': valid_loader
}

# loss function 설정
# criterion = nn.CrossEntropyLoss()
criterion = nn.L1Loss()

NameError: ignored

In [35]:
time_range=3
batch_size=64
train_loader, valid_loader, test_loader = create_classification_dataset([train_X_l, train_y], [valid_X_l, valid_y], 
                                                                        [test_X_l, test_y],time_range,batch_size)

# trining 단계에서 사용할 Dataloader dictionary 생성
dataloaders_dict = {
    'train': train_loader,
    'val': valid_loader
}

# loss function 설정
# criterion = nn.CrossEntropyLoss()
criterion = nn.L1Loss()
# LSTM 모델 1
lstm, lstm_val_acc_history = train_model(lstm, dataloaders_dict, criterion, 300,"LSTM",
                                         optimizer=optim.Adam(lstm.parameters(), lr=0.0001))

2
(22392, 3, 111)
Epoch 1/300
----------


TypeError: ignored

In [1]:
# GRU 모델 학습
gru, gru_val_acc_history = train_model(gru, dataloaders_dict, criterion, 300,"LSTM",
                                       optimizer=optim.Adam(gru.parameters(), lr=0.0001))

NameError: ignored

## Testing

In [ ]:
def test_model(model, test_loader):
    model.eval()   # 모델을 validation mode로 설정
    
    # test_loader에 대하여 검증 진행 (gradient update 방지)
    with torch.no_grad():
        running_loss = 0
        running_total = 0
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device, dtype=torch.float32)

            # forward
            # input을 model에 넣어 output을 도출
            outputs = model(inputs)

            # output 중 최댓값의 위치에 해당하는 class로 예측을 수행
            loss = criterion(outputs.to(torch.float32), labels.to(torch.float32))
            # batch별 정답 개수를 축적함
            running_loss += loss.item() * inputs.size(0)
            running_total += labels.size(0)


    # accuracy를 도출함
    test_acc = running_loss / running_total
    print('Testing Acc: {:4f}'.format(test_acc))
    
    return test_acc

In [ ]:
# LSTM 모델 검증 (Acc: 0.7333)
lstm2 = RNN(input_size, hidden_size, num_layers, bidirectional, rnn_type='lstm')
lstm2 = lstm2.to(device)

ckpt = torch.load('best_model_LSTM.pt')
lstm2.load_state_dict(ckpt)
test_model(lstm2, test_loader)

In [ ]:
# GRU 모델 검증 (Acc: 0.7333)
test_model( gru, test_loader)